<a href="https://colab.research.google.com/github/Mueble2/NLP-ChatBot-TA/blob/main/ChatBot_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🤖 **Chatbot Académico: Historia del Perú**

In [ ]:
!pip install sentence-transformers chromadb langchain langchain-community

In [23]:
# 📡 Carga de contenido web
import requests
from bs4 import BeautifulSoup
import re
# 🧠 Modelo de embedding (sentence transformers)
from sentence_transformers import SentenceTransformer
# 🧠 Embeddings y Vector Store
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from chromadb.config import Settings
import chromadb
# 📄 Procesamiento de documentos
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
# 🔍 Motor de recuperación de información
from langchain.chains import RetrievalQA
# 🤖 LLM vía Ollama
from langchain_community.llms import Ollama
#Para cargar documentos
from langchain_community.document_loaders import PyPDFLoader


In [24]:
def extraer_texto(url):
    try:
        resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
        resp.raise_for_status()  # Lanza excepción si falla (4xx/5xx)
    except requests.RequestException as e:
        print(f"❌ Error al acceder a {url}: {e}")
        return ""
    soup = BeautifulSoup(resp.text, "html.parser")
    # Elimina etiquetas no deseadas
    for tag in soup(["script", "style", "sup", "img", "figure", "table", "noscript"]):
        tag.decompose()
    # Extrae texto limpio
    texto = soup.get_text(separator=' ')
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

urls = [
    "https://es.wikipedia.org/wiki/Batalla_de_Ayacucho",
    "https://en.wikipedia.org/wiki/Battle_of_Ayacucho",
    "https://www.encyclopedia.com/humanities/encyclopedias-almanacs-transcripts-and-maps/ayacucho-battle",
    "https://www.britannica.com/topic/Battle-of-Ayacucho",
    "https://www.britannica.com/place/Ayacucho-Peru",
    "https://www.tierrasvivas.com/en/ayacucho-battle",
    "https://es.wikipedia.org/wiki/Capitulaci%C3%B3n_de_Ayacucho",
    "https://es.wikipedia.org/wiki/Santuario_hist%C3%B3rico_de_la_Pampa_de_Ayacucho",
    "https://elpais.com/america/2024-12-09/ayacucho-diciembre-9-1824-el-final-de-un-imperio-y-el-inicio-de-america-latina.html"
]

documentos = [extraer_texto(url) for url in urls]
print(f"✅ Se extrajeron {sum(1 for d in documentos if d)} documentos válidos.")

✅ Se extrajeron 9 documentos válidos.


In [25]:
# 1. Embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Chunking por oraciones
def dividir_en_chunks(texto, max_len=400):
    oraciones = re.split(r'(?<=[.?!])\s+', texto)
    chunks, actual = [], ""
    for oracion in oraciones:
        if len(actual) + len(oracion) <= max_len:
            actual += " " + oracion
        else:
            chunks.append(actual.strip())
            actual = oracion
    if actual:
        chunks.append(actual.strip())
    return chunks

# 3. Procesar todos los documentos
chunks = []
for doc in documentos:
    if doc:  # evita errores por documentos vacíos
        chunks.extend(dividir_en_chunks(doc))

# 4. Generar embeddings
vectores = embedding_model.encode(chunks)

# 5. Crear cliente Chroma
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))

# Crear colección nueva
collection = chroma_client.get_or_create_collection("batalla_ayacucho_rag")

# 6. Cargar datos
for i, chunk in enumerate(chunks):
    collection.add(documents=[chunk], ids=[f"chunk_{i}"], embeddings=[vectores[i].tolist()])

print(f"✅ {len(chunks)} chunks embebidos y cargados en Chroma.")

✅ 488 chunks embebidos y cargados en Chroma.


In [ ]:
#Descarga de OLlama
!curl -fsSL https://ollama.com/install.sh | sh  # Solo una vez

In [27]:
#Lanzar servidor de OLlama en local
!ollama serve > /dev/null 2>&1 &
!sleep 10

In [ ]:
#Descarga del modelo y librerías
!ollama pull phi3.5:latest
!pip install -U langchain-ollama

In [33]:
# Carga del modelo
from langchain_ollama import OllamaLLM
llm = OllamaLLM(
    model="phi3.5:latest",
    temperature=0.5,
    num_gpu=40,
    system=""
)

In [34]:
def responder_pregunta(pregunta: str, k=3, max_chars=1500) -> str:
    # Embedding de la pregunta
    query_emb = embedding_model.encode(pregunta).tolist()

    # Recuperar documentos similares
    resultados = collection.query(
        query_embeddings=[query_emb],
        n_results=k
    )

    # Construcción de contexto
    contextos = resultados["documents"][0]
    contexto = "\n\n".join(contextos)[:max_chars]

    # Prompt para Ollama + phi3.5
    prompt = f"""Responde en español a la siguiente pregunta sobre historia peruana.

  Contexto histórico:
  {contexto}

  Pregunta: {pregunta}
  Respuesta:"""

    # Llamada al modelo local
    return llm.invoke(prompt).strip()


In [36]:
pregunta = "¿Por qué se generó la Batalla de Ayacucho?"
respuesta = responder_pregunta(pregunta)

print("🧠 Pregunta:", pregunta)
print("📘 Respuesta:", respuesta)

🧠 Pregunta: ¿Por qué se generó la Batalla de Ayacucho?
📘 Respuesta: La Batalla de Ayacucho fue resultado de un combate civil entre dos bandos en el contexto del proceso independentista peruano. Según los testimonios histódicticos, tanto las fuerzas realistas como las patriotas contaron con auxiliares forasteros que jugaron un papel significativo durante la confrontación armada. La batalla se vio influenciada por acusaciones de traición y manipulación política según lo relatado por los oficiales españoles, mientras el Perú luchaba por su independencia del dominio español. Por tanto, Ayacucho representó un punto culminante en la guerra de liberación que buscaba desafiar las estructuras coloniales y asentar una nueva nación soberana.

La controversia sobre si el desenlace estaba preestablecido o no añade complejidad a nuestra comprensión histórica, pero es evidente que la lucha por la autonomía fue un factor central en la génesis de esta batalla decisiva para América Latina. La resistenci